In [19]:
import os
import random
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models, losses, optimizers, metrics

from model_train import (
    load_c3d_model,
    train_msupcl_model,
    linear_evaluation,
    load_c3d_sscl_model,
    train_simclr_model,
    linear_evaluation_sscl,
    supervised_contrastive_loss,
    nt_xent_loss,
)
from data_uniform_sup import VideoDataGenerator
from data_uniform_sscl import SSCLVideoDataGenerator
from paired_generator import PairedDataGenerator
from model_train_r2plus1d_18 import (
    load_r2plus1d_model,
    load_sscl_r2plus1d_model,
)


In [20]:
seed = 2042
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

input_shape = (12, 64, 64, 3)  # 数据生成器中定义的输入形状
num_classes = 2  # 有害内容或安全内容
feature_dim = 512
num_epochs = 3
batch_size = 4
temperature = 0.8
learning_rate = 0.001


In [21]:
# Define dataset paths
violence_negative_dir = './data/violence_dataset/NonViolence'
violence_positive_dir = './data/violence_dataset/Violence'
tiktok_negative_dir = './data/tiktok/train/Safe'
tiktok_positive_dir = './data/tiktok/train/Harmful Content'


In [22]:
# 定义函数用于采样视频
def sample_videos(directory, num_samples=100):
    all_videos = [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if f.endswith('.mp4')
    ]
    sampled_videos = random.sample(all_videos, min(num_samples, len(all_videos)))
    return sampled_videos


In [23]:
# Violence dataset
violence_negative_videos = sample_videos(violence_negative_dir, 50)
violence_positive_videos = sample_videos(violence_positive_dir, 50)

# TikTok dataset
tiktok_negative_videos = sample_videos(tiktok_negative_dir, 50)
tiktok_positive_videos = sample_videos(tiktok_positive_dir, 50)


In [24]:
def split_data(negative_videos, positive_videos, train_ratio=0.55, val_ratio=0.15):
    # 合并并打乱数据
    videos = negative_videos + positive_videos
    labels = [0] * len(negative_videos) + [1] * len(positive_videos)
    combined = list(zip(videos, labels))
    random.shuffle(combined)
    videos[:], labels[:] = zip(*combined)

    # 计算划分索引
    total = len(videos)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)

    # 划分数据集
    train_videos = videos[:train_end]
    train_labels = labels[:train_end]
    val_videos = videos[train_end:val_end]
    val_labels = labels[train_end:val_end]
    test_videos = videos[val_end:]
    test_labels = labels[val_end:]

    return (train_videos, train_labels), (val_videos, val_labels), (test_videos, test_labels)



In [25]:
# Violence dataset
(
    (violence_train_videos, violence_train_labels),
    (violence_val_videos, violence_val_labels),
    (violence_test_videos, violence_test_labels),
) = split_data(violence_negative_videos, violence_positive_videos)


# TikTok dataset
(
    (tiktok_train_videos, tiktok_train_labels),
    (tiktok_val_videos, tiktok_val_labels),
    (tiktok_test_videos, tiktok_test_labels),
) = split_data(tiktok_negative_videos, tiktok_positive_videos)


In [26]:

# Convert labels to numpy arrays and one-hot encode them if necessary
def prepare_labels(labels):
    return np.array(labels)


violence_train_labels_np = prepare_labels(violence_train_labels)
violence_val_labels_np = prepare_labels(violence_val_labels)
violence_test_labels_np = prepare_labels(violence_test_labels)


tiktok_train_labels_np = prepare_labels(tiktok_train_labels)
tiktok_val_labels_np = prepare_labels(tiktok_val_labels)
tiktok_test_labels_np = prepare_labels(tiktok_test_labels)

In [27]:
# Violence dataset generators
violence_train_generator = VideoDataGenerator(
    violence_train_videos,
    violence_train_labels_np,
    batch_size=batch_size,
    shuffle=True,
    augment=True,
)
violence_val_generator = VideoDataGenerator(
    violence_val_videos,
    violence_val_labels_np,
    batch_size=batch_size,
    shuffle=False,
)
violence_test_generator = VideoDataGenerator(
    violence_test_videos,
    violence_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
)

In [28]:
# TikTok dataset generators
tiktok_train_generator = VideoDataGenerator(
    tiktok_train_videos,
    tiktok_train_labels_np,
    batch_size=batch_size,
    shuffle=True,
    augment=True,
)
tiktok_val_generator = VideoDataGenerator(
    tiktok_val_videos,
    tiktok_val_labels_np,
    batch_size=batch_size,
    shuffle=False,
)
tiktok_test_generator = VideoDataGenerator(
    tiktok_test_videos,
    tiktok_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
)


In [29]:
# Load the model
base_model = load_c3d_model(input_shape=input_shape, feature_dim=feature_dim)
print("Base Model Summary:")
base_model.summary()

Base Model Summary:
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12, 64, 64, 3)]   0         
                                                                 
 conv3d_8 (Conv3D)           (None, 12, 64, 64, 64)    5248      
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 12, 32, 32, 64)   0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 12, 32, 32, 128)   221312    
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 6, 16, 16, 128)   0         
 3D)                                                             
                                                                 
 conv3d_10 (Conv3D)          (None, 6, 

In [30]:
def create_classification_model(base_model, num_classes):
    features = base_model.output
    outputs = layers.Dense(num_classes, activation='softmax')(features)
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

In [31]:
classification_model_violence = create_classification_model(base_model, num_classes)
# 冻结基础模型的参数
for layer in classification_model_violence.layers[:-1]:
    layer.trainable = False

classification_model_violence.compile(
    loss=losses.SparseCategoricalCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    metrics=[metrics.SparseCategoricalAccuracy()],
)


In [32]:
print("Classification Model for Violence Dataset Summary:")
classification_model_violence.summary()

Classification Model for Violence Dataset Summary:
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12, 64, 64, 3)]   0         
                                                                 
 conv3d_8 (Conv3D)           (None, 12, 64, 64, 64)    5248      
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 12, 32, 32, 64)   0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 12, 32, 32, 128)   221312    
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 6, 16, 16, 128)   0         
 3D)                                                             
                                                                 
 conv3d_

In [33]:
history_violence = classification_model_violence.fit(
    violence_train_generator,
    validation_data=violence_val_generator,
    epochs=num_epochs,
    
)

Epoch 1/3
13/13 [==============================] - 9s 637ms/step - loss: 0.6935 - sparse_categorical_accuracy: 0.4712 - val_loss: 0.6935 - val_sparse_categorical_accuracy: 0.4167
Epoch 2/3
13/13 [==============================] - 8s 620ms/step - loss: 0.6928 - sparse_categorical_accuracy: 0.5577 - val_loss: 0.6935 - val_sparse_categorical_accuracy: 0.4167
Epoch 3/3
13/13 [==============================] - 8s 580ms/step - loss: 0.6927 - sparse_categorical_accuracy: 0.5192 - val_loss: 0.6937 - val_sparse_categorical_accuracy: 0.4167


In [34]:
# Evaluate on Violence test set
base_c3d_results_violence = classification_model_violence.evaluate(violence_test_generator)
print(
    f"Violence Dataset - Test Loss: {base_c3d_results_violence[0]}, Test Accuracy: {base_c3d_results_violence[1]}"
)

7/7 [==============================] - 4s 492ms/step - loss: 0.6929 - sparse_categorical_accuracy: 0.5357
Violence Dataset - Test Loss: 0.6929368376731873, Test Accuracy: 0.5357142686843872


In [35]:
classification_model_tiktok = create_classification_model(base_model, num_classes)

for layer in classification_model_tiktok.layers[:-1]:
    layer.trainable = False
    
classification_model_tiktok.compile(
    loss=losses.SparseCategoricalCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    metrics=[metrics.SparseCategoricalAccuracy()],
)
print("Classification Model for TikTok Dataset Summary:")
classification_model_tiktok.summary()

Classification Model for TikTok Dataset Summary:
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 12, 64, 64, 3)]   0         
                                                                 
 conv3d_8 (Conv3D)           (None, 12, 64, 64, 64)    5248      
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 12, 32, 32, 64)   0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 12, 32, 32, 128)   221312    
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 6, 16, 16, 128)   0         
 3D)                                                             
                                                                 
 conv3d_10

In [36]:
history_tiktok = classification_model_tiktok.fit(
    tiktok_train_generator,
    validation_data=tiktok_val_generator,
    epochs=num_epochs,
)

Epoch 1/3
13/13 [==============================] - 25s 2s/step - loss: 0.6941 - sparse_categorical_accuracy: 0.3846 - val_loss: 0.6926 - val_sparse_categorical_accuracy: 0.5833
Epoch 2/3
13/13 [==============================] - 23s 2s/step - loss: 0.6935 - sparse_categorical_accuracy: 0.4519 - val_loss: 0.6933 - val_sparse_categorical_accuracy: 0.4167
Epoch 3/3
13/13 [==============================] - 23s 2s/step - loss: 0.6927 - sparse_categorical_accuracy: 0.5769 - val_loss: 0.6933 - val_sparse_categorical_accuracy: 0.4167


In [37]:

# Evaluate on TikTok test set
base_c3d_results_tiktok = classification_model_tiktok.evaluate(tiktok_test_generator)
print(
    f"TikTok Dataset - Test Loss: {base_c3d_results_tiktok[0]}, Test Accuracy: {base_c3d_results_tiktok[1]}"
)


7/7 [==============================] - 9s 1s/step - loss: 0.6931 - sparse_categorical_accuracy: 0.4643
TikTok Dataset - Test Loss: 0.693145751953125, Test Accuracy: 0.4642857015132904


## MSupCL implementation

In [38]:
# Combine training data from both datasets
combined_train_videos = violence_train_videos + tiktok_train_videos
combined_train_labels = violence_train_labels_np.tolist() + tiktok_train_labels_np.tolist()


# Create a combined data generator
combined_train_generator = VideoDataGenerator(
    combined_train_videos,
    combined_train_labels,
    batch_size=batch_size,
    shuffle=True,
    augment=True,
)

violence_train_generator_no_aug = VideoDataGenerator(
    violence_train_videos,
    violence_train_labels_np,
    batch_size=batch_size,
    shuffle=True,
    augment=False,
)

tiktok_train_generator_no_aug = VideoDataGenerator(
    tiktok_train_videos,
    tiktok_train_labels_np,
    batch_size=batch_size,
    shuffle=True,
    augment=False,
)

paired_train_generator = PairedDataGenerator(
    violence_train_generator_no_aug,
    tiktok_train_generator_no_aug,
    batch_size=16,
)

In [39]:
msupcl_model = load_c3d_model(input_shape=input_shape, feature_dim=feature_dim)
print("MSupCL Model Summary:")
msupcl_model.summary()

MSupCL Model Summary:
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 12, 64, 64, 3)]   0         
                                                                 
 conv3d_16 (Conv3D)          (None, 12, 64, 64, 64)    5248      
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 12, 32, 32, 64)   0         
 g3D)                                                            
                                                                 
 conv3d_17 (Conv3D)          (None, 12, 32, 32, 128)   221312    
                                                                 
 max_pooling3d_11 (MaxPoolin  (None, 6, 16, 16, 128)   0         
 g3D)                                                            
                                                                 
 conv3d_18 (Conv3D)          (None, 6

In [40]:

# Train the model
train_msupcl_model(msupcl_model, paired_train_generator, epochs=num_epochs, temperature=temperature)




Epoch 1/3
Training Loss: 0.6147
Epoch 2/3
Training Loss: 0.6121
Epoch 3/3
Training Loss: 0.6110


In [41]:
msupcl_c3d_result_violence, msupcl_c3d_result_tiktok = linear_evaluation(
    msupcl_model,
    combined_train_generator,
    violence_test_generator,
    tiktok_test_generator,
    num_classes=num_classes,
)


Epoch 1/5
27/27 [==============================] - 27s 977ms/step - loss: 2.6310 - accuracy: 0.4815
Epoch 2/5
27/27 [==============================] - 26s 973ms/step - loss: 1.8185 - accuracy: 0.4676
Epoch 3/5
27/27 [==============================] - 26s 956ms/step - loss: 1.6371 - accuracy: 0.5093
Epoch 4/5
27/27 [==============================] - 26s 968ms/step - loss: 1.9938 - accuracy: 0.4491
Epoch 5/5
27/27 [==============================] - 26s 978ms/step - loss: 1.5799 - accuracy: 0.5000
Evaluating on Violence Test Set:
7/7 [==============================] - 4s 494ms/step - loss: 0.6824 - accuracy: 0.5357
Violence Test Loss: 0.6823827624320984, Test Accuracy: 0.5357142686843872
Evaluating on TikTok Test Set:
7/7 [==============================] - 9s 1s/step - loss: 0.7261 - accuracy: 0.4643
TikTok Test Loss: 0.7260947227478027, Test Accuracy: 0.4642857015132904


## SSCL


In [42]:
violence_train_sscl_generator = SSCLVideoDataGenerator(
    violence_train_videos,
    violence_train_labels_np,
    batch_size=batch_size,
    shuffle=True,
    augment=True,
)

violence_val_sscl_generator = SSCLVideoDataGenerator(
    violence_val_videos,
    violence_val_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False,
    double_view=False,
)
violence_test_sscl_generator = SSCLVideoDataGenerator(
    violence_test_videos,
    violence_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False,
    double_view=False,
)
tiktok_train_sscl_generator = SSCLVideoDataGenerator(
    tiktok_train_videos,
    tiktok_train_labels_np,
    batch_size=batch_size,
    shuffle=True,
    augment=True,
)
tiktok_val_sscl_generator = SSCLVideoDataGenerator(
    tiktok_val_videos,
    tiktok_val_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False,
    double_view=False,
)
tiktok_test_sscl_generator = SSCLVideoDataGenerator(
    tiktok_test_videos,
    tiktok_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False,
    double_view=False,
)



In [43]:
sscl_model = load_c3d_sscl_model(input_shape=input_shape, feature_dim=feature_dim)
print("SSCL Model Summary:")
sscl_model.summary()

SSCL Model Summary:
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 12, 64, 64, 3)]   0         
                                                                 
 conv3d_24 (Conv3D)          (None, 12, 64, 64, 64)    5248      
                                                                 
 max_pooling3d_15 (MaxPoolin  (None, 12, 32, 32, 64)   0         
 g3D)                                                            
                                                                 
 conv3d_25 (Conv3D)          (None, 12, 32, 32, 128)   221312    
                                                                 
 max_pooling3d_16 (MaxPoolin  (None, 6, 16, 16, 128)   0         
 g3D)                                                            
                                                                 
 conv3d_26 (Conv3D)          (None, 6, 

In [44]:
# 训练SSCL模型（在暴力数据集上）
train_simclr_model(sscl_model, violence_train_sscl_generator, epochs=num_epochs, temperature=temperature)


Epoch 1/3


ValueError: Input 0 of layer "model_8" is incompatible with the layer: expected shape=(None, 12, 64, 64, 3), found shape=(4, 16, 112, 112, 3)

In [ ]:
# 在线性评估中使用训练好的SSCL模型（暴力数据集）
sscl_c3d_result_violence = linear_evaluation_sscl(
    sscl_model,
    violence_train_sscl_generator,
    violence_val_sscl_generator,
    violence_test_sscl_generator,
    num_classes=num_classes,
)

In [ ]:
train_simclr_model(sscl_model, tiktok_train_sscl_generator, epochs=num_epochs, temperature=temperature)


In [ ]:
sscl_c3d_result_tiktok = linear_evaluation_sscl(
    sscl_model,
    tiktok_train_sscl_generator,
    tiktok_val_sscl_generator,
    tiktok_test_sscl_generator,
    num_classes=num_classes,
)

## R2+1d_18 model

In [21]:
base_model = load_r2plus1d_model(input_shape=input_shape, feature_dim=feature_dim, include_top=False)

In [22]:
for layer in base_model.layers:
    layer.trainable = False
    
features = base_model.output
outputs = layers.Dense(num_classes, activation='softmax')(features)
classification_model = models.Model(inputs=base_model.input, outputs=outputs)

classification_model.compile(
    loss=losses.SparseCategoricalCrossentropy(),
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

In [23]:
# Train on Violence dataset
history_violence = classification_model.fit(
    violence_train_generator,
    validation_data=violence_val_generator,
    epochs=num_epochs
)

Epoch 1/10
27/27 [==============================] - 20s 744ms/step - loss: 0.6933 - sparse_categorical_accuracy: 0.4907 - val_loss: 0.6933 - val_sparse_categorical_accuracy: 0.4286
Epoch 2/10
27/27 [==============================] - 19s 691ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.4954 - val_loss: 0.6929 - val_sparse_categorical_accuracy: 0.5000
Epoch 3/10
27/27 [==============================] - 19s 679ms/step - loss: 0.6936 - sparse_categorical_accuracy: 0.4907 - val_loss: 0.6928 - val_sparse_categorical_accuracy: 0.6429
Epoch 4/10
27/27 [==============================] - 19s 687ms/step - loss: 0.6930 - sparse_categorical_accuracy: 0.5185 - val_loss: 0.6928 - val_sparse_categorical_accuracy: 0.5357
Epoch 5/10
27/27 [==============================] - 19s 684ms/step - loss: 0.6937 - sparse_categorical_accuracy: 0.5046 - val_loss: 0.6927 - val_sparse_categorical_accuracy: 0.5357
Epoch 6/10
27/27 [==============================] - 19s 698ms/step - loss: 0.6942 - sparse_cate

In [24]:
# Evaluate on Violence test set
base_r2plus1d_results_violence = classification_model.evaluate(violence_test_generator)
print(f"Violence Dataset - Test Loss: {base_r2plus1d_results_violence[0]}, Test Accuracy: {base_r2plus1d_results_violence[1]}")

15/15 [==============================] - 8s 556ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4833
Violence Dataset - Test Loss: 0.6932312250137329, Test Accuracy: 0.4833333194255829


In [25]:
history_tiktok = classification_model.fit(
    tiktok_train_generator,
    validation_data=tiktok_val_generator,
    epochs=num_epochs
)

Epoch 1/10
27/27 [==============================] - 60s 2s/step - loss: 0.6909 - sparse_categorical_accuracy: 0.5463 - val_loss: 0.6960 - val_sparse_categorical_accuracy: 0.3929
Epoch 2/10
27/27 [==============================] - 58s 2s/step - loss: 0.6892 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.7012 - val_sparse_categorical_accuracy: 0.3929
Epoch 3/10
27/27 [==============================] - 59s 2s/step - loss: 0.6866 - sparse_categorical_accuracy: 0.5556 - val_loss: 0.7015 - val_sparse_categorical_accuracy: 0.3929
Epoch 4/10
27/27 [==============================] - 58s 2s/step - loss: 0.6843 - sparse_categorical_accuracy: 0.5648 - val_loss: 0.7050 - val_sparse_categorical_accuracy: 0.3929
Epoch 5/10
27/27 [==============================] - 58s 2s/step - loss: 0.6828 - sparse_categorical_accuracy: 0.5648 - val_loss: 0.7045 - val_sparse_categorical_accuracy: 0.3929
Epoch 6/10
27/27 [==============================] - 58s 2s/step - loss: 0.6804 - sparse_categorical_accuracy: 

In [26]:
# Evaluate on TikTok test set
base_r2plus1d_results_tiktok = classification_model.evaluate(tiktok_test_generator)
print(f"TikTok Dataset - Test Loss: {base_r2plus1d_results_tiktok[0]}, Test Accuracy: {base_r2plus1d_results_tiktok[1]}")

15/15 [==============================] - 26s 2s/step - loss: 0.6990 - sparse_categorical_accuracy: 0.4500
TikTok Dataset - Test Loss: 0.6989744901657104, Test Accuracy: 0.44999998807907104


In [27]:

msupcl_model = load_r2plus1d_model(input_shape=input_shape, feature_dim=feature_dim, include_top=False)

In [28]:
msupcl_c3d_result_violence, msupcl_c3d_result_tiktok = linear_evaluation(
    msupcl_model,
    combined_train_generator,
    violence_test_generator,
    tiktok_test_generator,
    num_classes=num_classes,
)

violence_train_generator_no_aug = VideoDataGenerator(violence_train_videos, violence_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)
tiktok_train_generator_no_aug = VideoDataGenerator(tiktok_train_videos, tiktok_train_labels_np, batch_size=batch_size, shuffle=True, augment=False)
paired_train_generator = PairedDataGenerator(violence_train_generator_no_aug, tiktok_train_generator_no_aug)

In [29]:
train_msupcl_model(msupcl_model, paired_train_generator, epochs=num_epochs)

Epoch 1/10
Training Loss: 0.6702
Epoch 2/10
Training Loss: 0.6660
Epoch 3/10
Training Loss: 0.6695
Epoch 4/10
Training Loss: 0.6684
Epoch 5/10
Training Loss: 0.6684
Epoch 6/10
Training Loss: 0.6688
Epoch 7/10
Training Loss: 0.6676
Epoch 8/10
Training Loss: 0.6655
Epoch 9/10
Training Loss: 0.6670
Epoch 10/10
Training Loss: 0.6648


In [30]:
msupcl_r2plus1d_result_violence, msupcl_r2plus1d_result_tiktok = linear_evaluation(msupcl_model, combined_train_generator, violence_test_generator,tiktok_test_generator, num_classes=num_classes)

Epoch 1/5
55/55 [==============================] - 74s 1s/step - loss: 0.8126 - accuracy: 0.5341 - val_loss: 0.7668 - val_accuracy: 0.4500
Epoch 2/5
55/55 [==============================] - 72s 1s/step - loss: 0.7003 - accuracy: 0.5341 - val_loss: 0.7598 - val_accuracy: 0.5833
Epoch 3/5
55/55 [==============================] - 72s 1s/step - loss: 0.6988 - accuracy: 0.5705 - val_loss: 0.7940 - val_accuracy: 0.4500
Epoch 4/5
55/55 [==============================] - 72s 1s/step - loss: 0.7092 - accuracy: 0.5477 - val_loss: 0.7422 - val_accuracy: 0.4667
Epoch 5/5
55/55 [==============================] - 72s 1s/step - loss: 0.7077 - accuracy: 0.5636 - val_loss: 0.7156 - val_accuracy: 0.4833
Evaluating on Violence Test Set:
15/15 [==============================] - 8s 553ms/step - loss: 0.7526 - accuracy: 0.4500
Violence Test Loss: 0.7525920867919922, Test Accuracy: 0.44999998807907104
Evaluating on TikTok Test Set:
15/15 [==============================] - 26s 2s/step - loss: 0.7540 - accurac

# SSCL 

In [10]:


violence_train_sscl_generator = SSCLVideoDataGenerator(
    violence_train_videos,
    violence_train_labels_np,
    batch_size=batch_size, 
    shuffle=True, 
    augment=True
)

violence_val_single_sscl_generator = SSCLVideoDataGenerator(
    violence_val_videos,
    violence_val_labels_np,
    batch_size=batch_size, 
    shuffle=True, 
    augment=True, 
    double_view=False
)


violence_test_sscl_generator = SSCLVideoDataGenerator(
    violence_test_videos,
    violence_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False, 
    double_view=False
)

tiktok_train_sscl_generator = SSCLVideoDataGenerator(
    tiktok_train_videos,
    tiktok_train_labels_np,
    batch_size=batch_size, 
    shuffle=True, 
    augment=True
)

tiktok_val_single_sscl_generator = SSCLVideoDataGenerator(
    tiktok_val_videos,
    tiktok_val_labels_np,
    batch_size=batch_size, 
    shuffle=True, 
    augment=True, 
    double_view=False
)

tiktok_test_sscl_generator = SSCLVideoDataGenerator(
    tiktok_test_videos,
    tiktok_test_labels_np,
    batch_size=batch_size,
    shuffle=False,
    augment=False, 
    double_view=False
)

violence_train_single_sscl_generator = SSCLVideoDataGenerator(violence_train_videos,violence_train_labels_np,batch_size=batch_size, shuffle=True, augment=True, double_view=False)


tiktok_train_single_sscl_generator = SSCLVideoDataGenerator(tiktok_train_videos,tiktok_train_labels_np,batch_size=batch_size, shuffle=True, augment=True, double_view=False)


In [31]:


model = load_c3d_sscl_model(input_shape=input_shape, feature_dim=feature_dim)

train_simclr_model(model, violence_train_sscl_generator, epochs=num_epochs)

Epoch 1/10
Epoch 1, Loss: 1.9161
Epoch 2/10
Epoch 2, Loss: 1.9715
Epoch 3/10
Epoch 3, Loss: 1.9399
Epoch 4/10
Epoch 4, Loss: 1.9738
Epoch 5/10
Epoch 5, Loss: 1.9459
Epoch 6/10
Epoch 6, Loss: 1.9459
Epoch 7/10
Epoch 7, Loss: 1.9459
Epoch 8/10
Epoch 8, Loss: 1.9459
Epoch 9/10
Epoch 9, Loss: 1.9459
Epoch 10/10
Epoch 10, Loss: 1.9459


In [32]:


sscl_c3d_result_violence = linear_evaluation_sscl(model, violence_train_single_sscl_generator, violence_val_single_sscl_generator,violence_test_sscl_generator, num_classes=num_classes)

Epoch 1/5
27/27 [==============================] - 66s 2s/step - loss: 0.6932 - accuracy: 0.4907 - val_loss: 0.6932 - val_accuracy: 0.4643
Epoch 2/5
27/27 [==============================] - 61s 2s/step - loss: 0.6932 - accuracy: 0.4815 - val_loss: 0.6931 - val_accuracy: 0.5357
Epoch 3/5
27/27 [==============================] - 64s 2s/step - loss: 0.6932 - accuracy: 0.4630 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/5
27/27 [==============================] - 62s 2s/step - loss: 0.6932 - accuracy: 0.4815 - val_loss: 0.6933 - val_accuracy: 0.4643
Epoch 5/5
15/15 [==============================] - 27s 2s/step - loss: 0.6932 - accuracy: 0.4833
Test Loss: 0.6931995749473572, Test Accuracy: 0.4833333194255829


In [33]:



train_simclr_model(model, tiktok_train_sscl_generator, epochs=num_epochs, temperature=temperature)
sscl_c3d_result_tiktok = linear_evaluation_sscl(model, tiktok_train_single_sscl_generator,tiktok_val_single_sscl_generator,tiktok_test_sscl_generator, num_classes=num_classes)

Epoch 1/10
Epoch 1, Loss: 1.9459
Epoch 2/10
Epoch 2, Loss: 1.9459
Epoch 3/10
Epoch 3, Loss: 1.9459
Epoch 4/10
Epoch 4, Loss: 1.9459
Epoch 5/10
Epoch 5, Loss: 1.9459
Epoch 6/10
Epoch 6, Loss: 1.9459
Epoch 7/10
Epoch 7, Loss: 1.9459
Epoch 8/10
Epoch 8, Loss: 1.9459
Epoch 9/10
Epoch 9, Loss: 1.9459
Epoch 10/10
Epoch 10, Loss: 1.9459
Epoch 1/5
27/27 [==============================] - 183s 7s/step - loss: 0.6930 - accuracy: 0.6019 - val_loss: 0.6933 - val_accuracy: 0.4286
Epoch 2/5
27/27 [==============================] - 181s 7s/step - loss: 0.6931 - accuracy: 0.5648 - val_loss: 0.6937 - val_accuracy: 0.3929
Epoch 3/5
27/27 [==============================] - 180s 7s/step - loss: 0.6929 - accuracy: 0.5556 - val_loss: 0.6936 - val_accuracy: 0.4286
Epoch 4/5
27/27 [==============================] - 179s 7s/step - loss: 0.6928 - accuracy: 0.5463 - val_loss: 0.6939 - val_accuracy: 0.3929
Epoch 5/5
15/15 [==============================] - 81s 5s/step - loss: 0.6936 - accuracy: 0.4500
Test Loss: 

In [13]:
model = load_sscl_r2plus1d_model(input_shape=input_shape, feature_dim=feature_dim)
train_simclr_model(model, violence_train_sscl_generator, epochs=num_epochs, temperature=temperature)

Epoch 1/10
Epoch 1, Loss: 1.5959
Epoch 2/10
Epoch 2, Loss: 1.5192
Epoch 3/10
Epoch 3, Loss: 1.4695
Epoch 4/10
Epoch 4, Loss: 1.2889
Epoch 5/10
Epoch 5, Loss: 1.2568
Epoch 6/10
Epoch 6, Loss: 1.4238
Epoch 7/10
Epoch 7, Loss: 1.2884
Epoch 8/10
Epoch 8, Loss: 1.2220
Epoch 9/10
Epoch 9, Loss: 1.2543
Epoch 10/10
Epoch 10, Loss: 1.1313


In [35]:

sscl_r2plus1d_result_violence = linear_evaluation_sscl(model, violence_train_single_sscl_generator, violence_val_single_sscl_generator,violence_test_sscl_generator, num_classes=num_classes)

Epoch 1/5
27/27 [==============================] - 67s 3s/step - loss: nan - accuracy: 0.5093 - val_loss: nan - val_accuracy: 0.5357
Epoch 2/5
27/27 [==============================] - 64s 2s/step - loss: nan - accuracy: 0.5093 - val_loss: nan - val_accuracy: 0.5000
Epoch 3/5
27/27 [==============================] - 61s 2s/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5357
Epoch 4/5
27/27 [==============================] - 62s 2s/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5357
Epoch 5/5
15/15 [==============================] - 28s 2s/step - loss: nan - accuracy: 0.4833
Test Loss: nan, Test Accuracy: 0.4833333194255829


In [36]:
train_simclr_model(model, tiktok_train_sscl_generator, epochs=num_epochs, temperature=temperature)
sscl_r2plus1d_result_tiktok = linear_evaluation_sscl(model, tiktok_train_single_sscl_generator,tiktok_val_single_sscl_generator,tiktok_test_sscl_generator, num_classes=num_classes)

Epoch 1/10
Epoch 1, Loss: nan
Epoch 2/10


KeyboardInterrupt: 